## Library imports

In [ ]:
import numpy as np
import pandas as pd
from chromatography import ExperimentAnalytes
from separation_utility import *
from torch import optim, tensor
import matplotlib.pyplot as plt
import time

## Dataset Import

In [ ]:
alists = []
alists.append(pd.read_csv(f'../data/GilarSample.csv'))
alists.append(pd.read_csv(f'../data/Alizarin.csv'))
alists.append(pd.read_csv(f'../data/Peterpeptides.csv'))
alists.append(pd.read_csv(f'../data/Roca.csv'))
alists.append(pd.read_csv(f'../data/Peter32.csv'))
alists.append(pd.read_csv(f'../data/Eosin.csv'))
alists.append(pd.read_csv(f'../data/Controlmix2.csv'))
# GilarSample - 8 analytes
# Peterpeptides - 32 analytes
# Roca - 14 analytes
# Peter32 - 32 analytes
# Eosin - 20 analytes
# Alizarin - 16 analytes
# Controlmix2 - 17 analytes
# Gooding - 872 analytes

## Useful functions definition

In [ ]:
def loss_field(exp, taus, N = 200):
    phis = np.linspace(0, 1, N)
    losses = np.zeros((N, N))
    j = 0
    for phi1 in phis:
        i = 0
        for phi2 in phis:
            exp.reset()
            exp.run_all([phi1, phi2], taus)
            losses[i, j] = exp.loss()
            i += 1
        j += 1
    X, Y = np.meshgrid(phis, phis)
    
    return X, Y, losses
def average_over_equal_intervals(arr, interval):
    return np.mean(arr.reshape(-1, interval), axis=1)

### Experiment: Performance vs n_steps

In [ ]:
# Parameters
sigma_max = 0.3

kwargs = {
'num_episodes' : 6000,
'sample_size' : 10,
'lr' : .05,
'optim' : torch.optim.SGD,
'lr_decay_factor' : .5,
'lr_milestones' : 1000,
'print_every' : 6001,
'baseline' : 0.55,
'max_norm' : 2.
}
N = 10
M = 50
# Experiments
exp_8 = ExperimentAnalytes(k0 = alists[0].k0.values, S = alists[0].S.values, h=0.001, run_time=1.0)
exp_16 = ExperimentAnalytes(k0 = alists[1].k0.values, S = alists[1].S.values, h=0.001, run_time=1.0)
exp_32 = ExperimentAnalytes(k0 = alists[2].k0.values, S = alists[2].S.values, h=0.001, run_time=1.0)
# Final Results 
dist_8 = np.zeros((N, M))
dist_16 = np.zeros((N, M))
dist_32 = np.zeros((N, M))

for n in range(0, N):
    print(n)
    delta_taus = np.ones(n + 1) * 1/(n + 1)
    
    for i in range(M):
        print(f"  {i}")
        #Policies
        pol_8 = PolicySingle(len(delta_taus), sigma_max = sigma_max)
        pol_16 = PolicySingle(len(delta_taus), sigma_max = sigma_max)
        pol_32 = PolicySingle(len(delta_taus), sigma_max = sigma_max)
        # Run Exp 8
        reinforce_one_set(
                exp_8, 
                pol_8, 
                delta_taus=delta_taus, 
                **kwargs
            )
        # Run Exp 16
        reinforce_one_set(
                exp_16, 
                pol_16, 
                delta_taus=delta_taus, 
                **kwargs
            )
        # Run Exp 32
        reinforce_one_set(
                exp_32, 
                pol_32, 
                delta_taus=delta_taus, 
                **kwargs
            )
        
        exp_8.reset()
        exp_16.reset()
        exp_32.reset()
        mu_8, _ = pol_8.forward()
        mu_16, _ = pol_16.forward()
        mu_32, _ = pol_32.forward()
        exp_8.run_all(mu_8.detach().numpy(), delta_taus)
        exp_16.run_all(mu_16.detach().numpy(), delta_taus)
        exp_32.run_all(mu_32.detach().numpy(), delta_taus)
        
        dist_8[n, i] = exp_8.loss()
        dist_16[n, i] = exp_16.loss()
        dist_32[n, i] = exp_32.loss()

In [ ]:
np.savez_compressed("../results/distribution_single_perf_vs_n_steps", nr_analytes_8=dist_8, nr_analytes_16=dist_16, nr_analytes_32=dist_32)

### Experiment: RL vs grid search

In [ ]:
# Parameters
sigma_max = 0.3

kwargs = {
'num_episodes' : 6000,
'sample_size' : 10,
'lr' : .05,
'optim' : torch.optim.SGD,
'lr_decay_factor' : .5,
'lr_milestones' : 1000,
'print_every' : 6001,
'baseline' : 0.55,
'max_norm' : 2.
}
exp = ExperimentAnalytes(k0 = alists[2].k0.values, S = alists[2].S.values, h=0.001, run_time=1.0)

### one n_step
loss = 3.
delta_taus = [1.]
print("Grid Search 1")
# Grid Search
start_gs_1 = time.perf_counter()
for phi in np.linspace(0, 1, 100):
    exp.reset()
    exp.run_all([phi], delta_taus)
    if exp.loss() < loss:
        loss = exp.loss()
        best_1 = [phi]
end_gs_1 = time.perf_counter()
# RL
print("RL 1")
start_rl_1 = time.perf_counter()
for n in range(10):
    pol = PolicySingle(len(delta_taus), sigma_max = sigma_max)
    reinforce_one_set(
            exp, 
            pol, 
            delta_taus=delta_taus, 
            **kwargs
        )
end_rl_1 = time.perf_counter()


### two n_step
loss = 3.
delta_taus = [.5, .5]
# Grid Search
print("Grid Search 2")
start_gs_2 = time.perf_counter()
for phi_1 in np.linspace(0, 1, 100):
    for phi_2 in np.linspace(0, 1, 100):
        exp.reset()
        exp.run_all([phi_1, phi_2], delta_taus)
        if exp.loss() < loss:
            loss = exp.loss()
            best_2 = [phi_1, phi_2]
end_gs_2 = time.perf_counter()
# RL
print("RL 2")
start_rl_2 = time.perf_counter()
for n in range(10):
    pol = PolicySingle(len(delta_taus), sigma_max = sigma_max)
    reinforce_one_set(
            exp, 
            pol, 
            delta_taus=delta_taus, 
            **kwargs
        )
end_rl_2 = time.perf_counter()
### three n_step
loss = 3.
delta_taus = [.33, .33, .34]
# Grid Search
print("Grid Search 3")
start_gs_3 = time.perf_counter()
for phi_1 in np.linspace(0, 1, 100):
    for phi_2 in np.linspace(0, 1, 100):
        for phi_3 in np.linspace(0, 1, 100):
            exp.reset()
            exp.run_all([phi_1, phi_2, phi_3], delta_taus)
            if exp.loss() < loss:
                loss = exp.loss()
                best_3 = [phi_1, phi_2, phi_3]
end_gs_3 = time.perf_counter()
# RL
print("RL 3")
start_rl_3 = time.perf_counter()
for n in range(10):
    pol = PolicySingle(len(delta_taus), sigma_max = sigma_max)
    reinforce_one_set(
            exp, 
            pol, 
            delta_taus=delta_taus, 
            **kwargs
        )
end_rl_3 = time.perf_counter()
### four n_step
loss = 3.
delta_taus = [.25, .25, .25, .25]
# Grid Search
print("Grid Search 4")
start_gs_4 = time.perf_counter()
for phi_1 in np.linspace(0, 1, 100):
    for phi_2 in np.linspace(0, 1, 100):
        for phi_3 in np.linspace(0, 1, 100):
            for phi_4 in np.linspace(0, 1, 100):
                exp.reset()
                exp.run_all([phi_1, phi_2, phi_3, phi_4], delta_taus)
                if exp.loss() < loss:
                    loss = exp.loss()
                    best_4 = [phi_1, phi_2, phi_3, phi_4]
end_gs_4 = time.perf_counter()
# RL
print("RL 4")
start_rl_4 = time.perf_counter()
for n in range(10):
    pol = PolicySingle(len(delta_taus), sigma_max = sigma_max)
    reinforce_one_set(
            exp, 
            pol, 
            delta_taus=delta_taus, 
            **kwargs
        )
end_rl_4 = time.perf_counter()

grid_search_time_ms = np.array([end_gs_1 - start_gs_1, end_gs_2 - start_gs_2, end_gs_3 - start_gs_3, end_gs_4 - start_gs_4])
rl_time_ms = np.array([end_rl_1 - start_rl_1, end_rl_2 - start_rl_2, end_rl_3 - start_rl_3, end_rl_4 - start_rl_4])/10

In [ ]:
np.savetxt("../results/grid_search_time_ms.txt", grid_search_time_ms)
np.savetxt("../results/rl_time_ms.txt", rl_time_ms)

In [ ]:
print(best_1)
print(best_2)
print(best_3)
print(best_4)
[0.3535353535353536]
[0.31313131313131315, 0.38383838383838387]
[0.32323232323232326, 0.38383838383838387, 0.37373737373737376]
[0.32323232323232326, 0.37373737373737376, 0.38383838383838387, 0.37373737373737376]

### Experiment: RL vs delta tau RL

In [ ]:
# Parameters
sigma_max = 0.3

kwargs = {
'num_episodes' : 6000,
'sample_size' : 10,
'lr' : .05,
'optim' : torch.optim.SGD,
'lr_decay_factor' : .5,
'lr_milestones' : 1000,
'print_every' : 6001,
'baseline' : 0.55,
'max_norm' : 2.
}
N = 10
M = 50
# Experiments
exp_8 = ExperimentAnalytes(k0 = alists[0].k0.values, S = alists[0].S.values, h=0.001, run_time=1.0)
exp_16 = ExperimentAnalytes(k0 = alists[1].k0.values, S = alists[1].S.values, h=0.001, run_time=1.0)
exp_32 = ExperimentAnalytes(k0 = alists[2].k0.values, S = alists[2].S.values, h=0.001, run_time=1.0)
# Final Results 
dist_tau_8 = np.zeros((N, M))
dist_tau_16 = np.zeros((N, M))
dist_tau_32 = np.zeros((N, M))

for n in range(1, N):    
    for i in range(M):
        print(f"  {i}")
        #Policies
        pol_tau_8 = PolicySingleTime(n + 1, sigma_max = sigma_max)
        pol_tau_16 = PolicySingleTime(n + 1, sigma_max = sigma_max)
        pol_tau_32 = PolicySingleTime(n + 1, sigma_max = sigma_max)
        # Run Exp 8
        reinforce_delta_tau(
                exp_8, 
                pol_tau_8,
                **kwargs
            )
        # Run Exp 16
        reinforce_delta_tau(
                exp_16, 
                pol_tau_16,
                **kwargs
            )
        # Run Exp 32
        reinforce_delta_tau(
                exp_32, 
                pol_tau_32,
                **kwargs
            )
        
        exp_8.reset()
        exp_16.reset()
        exp_32.reset()
        mu_8, _ = pol_tau_8.forward()
        mu_16, _ = pol_tau_16.forward()
        mu_32, _ = pol_tau_32.forward()
        grads_8, delta_taus_8 = np.split(torch.cat((mu_8, tensor([1.])), 0).data.numpy(), 2)
        grads_16, delta_taus_16 = np.split(torch.cat((mu_16, tensor([1.])), 0).data.numpy(), 2)
        grads_32, delta_taus_32 = np.split(torch.cat((mu_32, tensor([1.])), 0).data.numpy(), 2)
        exp_8.run_all(grads_8, delta_taus_8)
        exp_16.run_all(grads_16, delta_taus_16)
        exp_32.run_all(grads_32, delta_taus_32)
        
        dist_tau_8[n, i] = exp_8.loss()
        dist_tau_16[n, i] = exp_16.loss()
        dist_tau_32[n, i] = exp_32.loss()

In [ ]:
np.savez_compressed("../results/distribution_single_delta_tau_perf_vs_n_steps", nr_analytes_8=dist_tau_8, nr_analytes_16=dist_tau_16, nr_analytes_32=dist_tau_32)